## Задача 2

### Решить систему, реализовав итерационные методы, указанные в варианте (варианты - 12.7 по методичке, стр.101, матрицы - из предыдущего задания, 11.6, стр.94).
### Вариант 7

$
A =
\begin{pmatrix}
    9.331343 & 1.120045 & -2.880925 \\
    1.120045 & 7.086042 & 0.670297 \\
    -2.880925 & 0.670297 & 5.622534
\end{pmatrix}
$

$
b =
\begin{pmatrix}
    7.570463 \\
    8.876384 \\
    3.411906
\end{pmatrix}
$

$k = 7$

Входные данные

In [1]:
import maths
import numpy as np
from math import sqrt

A = np.array([
    [9.331343, 1.120045, -2.880925],
    [1.120045, 7.086042, 0.670297],
    [-2.880925, 0.670297, 5.622534]
], dtype=np.float64)

b = np.array([7.570463, 8.876384, 3.411906], dtype=np.float64).reshape([3, 1])

k = 7

Решение методом Гаусса

In [2]:
correct_x = maths.gaussian_method(A, b)
correct_x

array([[1.],
       [1.],
       [1.]])

Нахождение $H_d$, $g_d$ и $\|H_d\|_\infty$

In [3]:
diagonals_A = np.diag(np.diag(A))
inv_diagonals_A = np.linalg.inv(diagonals_A)
H_d = np.eye(A.shape[0], dtype=A.dtype) - inv_diagonals_A.dot(A)

norm_H_d = np.linalg.norm(H_d, ord=np.inf)

g_d = inv_diagonals_A.dot(b)

norm_g_d = np.linalg.norm(g_d, ord=np.inf)
print(f"H_d = \n{H_d}\n")
print(f"g_d = \n{g_d}\n")
print(f"norm(H_d) = {norm_H_d}")

H_d = 
[[ 0.         -0.12003042  0.30873637]
 [-0.15806356  0.         -0.09459399]
 [ 0.51238908 -0.11921618  0.        ]]

g_d = 
[[0.81129404]
 [1.25265755]
 [0.6068271 ]]

norm(H_d) = 0.6316052512977246


Априорная оценка

In [4]:
apriori_estimate = (norm_H_d ** k) * norm_g_d / (1 - norm_H_d)
apriori_estimate

0.13634494907408518

Метод простой итерации

In [5]:
def simple_iteration_method(H: np.ndarray, g: np.ndarray, count: int):
    prev_x = np.zeros(g.shape, dtype=g.dtype)
    x = prev_x.copy()
    for i in range(count):
        prev_x = x.copy()
        x = H.dot(prev_x) + g
    return prev_x, x

Решение методом простых итераций

In [6]:
(prev_simple_x, simple_x) = simple_iteration_method(H_d, g_d, k)
simple_x

array([[0.9976332 ],
       [1.00160563],
       [0.99634648]])

Апостериорная оценка

In [7]:
aposterior_estimate = norm_H_d / (1 - norm_H_d) * np.linalg.norm(simple_x - prev_simple_x, ord=np.inf)
aposterior_estimate

0.006829710518038748

Фактическая погрешность метода простых итераций

In [8]:
fact_simple_error = np.abs(correct_x - simple_x)
fact_simple_error

array([[0.0023668 ],
       [0.00160563],
       [0.00365352]])

Уточнение последнего приближения по Люстернику

In [9]:
spectral_radius_H_d = np.abs(np.linalg.eigvals(H_d)).max()
luster_x = prev_simple_x + 1 / (1 - spectral_radius_H_d) * (simple_x - prev_simple_x)
luster_x

array([[1.0010566 ],
       [1.00010442],
       [0.99867472]])

Фактическая погрешность

In [10]:
fact_luster_error = np.abs(correct_x - luster_x)
fact_luster_error

array([[0.0010566 ],
       [0.00010442],
       [0.00132528]])

Метод Зейделя

In [11]:
def seidel_method(H: np.ndarray, g: np.ndarray, count: int):
    H_l = H.copy()
    H_r = H.copy()
    for i in range(H.shape[0]):
        for j in range(H.shape[1]):
            if i < j:
                H_l[i, j] = 0
            else:
                H_r[i, j] = 0

    H_seid = np.linalg.inv(np.eye(H.shape[0], dtype=H.dtype) - H_l).dot(H_r)
    g_seid = np.linalg.inv(np.eye(H.shape[0], dtype=H.dtype) - H_l).dot(g)
    H_seid_spectral_radius = np.abs(np.linalg.eigvals(H_seid)).max()

    prev_x = np.zeros(g.shape, dtype=g.dtype)
    x = prev_x.copy()
    for i in range(count):
        prev_x = x.copy()
        x = H_seid.dot(prev_x) + g_seid

    return x, H_seid_spectral_radius

Решение методом Зейделя

In [12]:
(seidel_x, seidel_spectral_radius) = seidel_method(H_d, g_d, k)
seidel_x

array([[0.99997462],
       [1.0000101 ],
       [0.99998579]])

Фактическая погрешность метода Зейделя

In [13]:
seidel_fact_error = np.abs(correct_x - seidel_x)

Спектральный радиус матрицы перехода метода Зейделя

In [14]:
seidel_spectral_radius

0.22062932793191173

Метод верхней релаксации

In [15]:
def upper_relaxations(H: np.ndarray, g: np.ndarray, count: int):
    H_spectral_radius = np.abs(np.linalg.eigvals(H)).max()
    q_opt = 2 / (1 +  sqrt(1 - H_spectral_radius ** 2))

    x = np.zeros(g.shape, dtype=g.dtype)
    n = H.shape[0]
    for cnt in range(count):
        prev_x = x.copy()
        for i in range(n):
            temp = -prev_x[i, 0] + g[i, 0]
            for j in range(n):
                if j <= i - 1:
                    temp += H[i, j] * x[j, 0]
                if j >= i + 1:
                    temp += H[i, j] * prev_x[j, 0]
            x[i, 0] = prev_x[i, 0] + q_opt * temp
    return x

Решение методом верхней релаксации

In [16]:
upper_relaxations_x = upper_relaxations(H_d, g_d, k)
upper_relaxations_x

array([[0.99999973],
       [1.00000015],
       [0.99999991]])

Фактическая погрешность метода верхней релаксации

In [17]:
upper_relaxations_fact_error = np.abs(correct_x - upper_relaxations_x)
upper_relaxations_fact_error

array([[2.67080491e-07],
       [1.53320792e-07],
       [8.85392536e-08]])